#Mount google drive

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    base_path = "/content/gdrive/My Drive"
except:
    base_path = "C:/Users/Schiller/Google Drive/"
%cd {base_path}

import os
trained_dir     = base_path + "/trained_models"
model_name      = "efficientdet-d0_2020_11_10-16"
input_saved_model_dir   = os.path.join(trained_dir, model_name, "saved_model")
output_saved_model_dir  = os.path.join(trained_dir, model_name, "tensorRT")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive


In [2]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

params = tf.experimental.tensorrt.ConversionParams(
    precision_mode='FP16')
converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir=input_saved_model_dir, conversion_params=params)
converter.convert()
converter.save(output_saved_model_dir)

os.listdir(output_saved_model_dir)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: /content/gdrive/My Drive/trained_models/efficientdet-d0_2020_11_10-16/tensorRT/assets


['variables', 'assets', 'saved_model.pb']

In [3]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(
    max_workspace_size_bytes=(1<<32))
conversion_params = conversion_params._replace(precision_mode="FP16")
conversion_params = conversion_params._replace(
    maximum_cached_engines=100)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=input_saved_model_dir,
    conversion_params=conversion_params)
converter.convert()

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)


In [6]:
import numpy as np
def my_input_fn():
# Input for a single inference call, for a network that has two input tensors:
  Inp1 = np.random.normal(size=(1, 255, 255, 3)).astype(np.uint8)
  yield (Inp1,)
  """
converter.build(input_fn=my_input_fn)
converter.save(output_saved_model_dir)
"""
saved_model_loaded = tf.saved_model.load(
    output_saved_model_dir, tags=[tag_constants.SERVING])
graph_func = saved_model_loaded.signatures[
    signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
frozen_func = convert_to_constants.convert_variables_to_constants_v2(
    graph_func)
output = frozen_func(input_data)[0].numpy()

NameError: ignored